# Homework 2: Text Classification

GIT SANITY CHECK
PYCHARM NOT CHECKING IN JUPYTER CHANGES

Text classification is a fundamental NLP task. In this assignment, you will build and evaluate a simple text classification pipeline, using both from-scratch as well as library implementations of standard classification algorithms.

This problem set is adapted from the first problem set given in Jacob Eisentstein's Spring Term 2018 course in NLP, taught at Georgia Tech, and is modified from [the original](https://github.com/jacobeisenstein/gt-nlp-class/blob/master/psets/ps1/pset1.ipynb) with permission of the author.

## 0. Setup

You will need to install several Python libraries in order to do this assignment. They are all installable with either `anaconda` or `pip`.

* jupyter
* matplotlib
* sklearn
* nose
* pandas

Most of your coding for this assignment will be in files in the `hw2_utils` package as well as in this Jupyter notebook. I have also provided automated unit tests which will help you determine if your code is working correctly, located in the `tests` directory. You are free to examine the code for the unit tests- in fact, you _should_ examine the code, as you will likely find it helpful. Note that these tests are the same ones that I will be using to help grade the assignment, so one of your goals with this assignment is to get all of the tests to pass. For more information about working with unit tests, see [this guide](http://pythontesting.net/framework/nose/nose-introduction/).

Note that the skeleton function implementations in `hw2_utils` all have careful and precise docstring annotations indicating types of parameters and returns. Pay careful attention to these, as you will be using your own functions elsewhere in the assignment!

If you have never used Jupyter notebooks before, [this guide](https://www.dataquest.io/blog/jupyter-notebook-tutorial/) may be useful.

### Verification & loading of packages, etc.

In [65]:
import sys
from importlib import reload

If you do not already have the relevant packages installed, execute the following cell:

In [66]:
!pip3 install sklearn nose matplotlib pandas

In [67]:
import nose
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

What versions do you have installed?

In [68]:
print(sys.version)
print("My library versions:")
print(f"Pandas: {pd.__version__}")
print(f"Numpy: {np.__version__}")
print(f"Scipy: {sp.__version__}")
print(f"matplotlib: {matplotlib.__version__}")
print(f"nose: {nose.__version__}")

Let's run our first unit test, to make sure that your environment is set up correctly:

In [69]:
! nosetests tests_instructor/test_environment.py

## 1. Preprocessing

We'll be using Pandas dataframes to do some of our work. Start by reading a chunk of the data into a dataframe:

In [70]:

df_train = pd.read_csv('lyrics-train.csv')
df_dev = pd.read_csv('lyrics-dev.csv')
df_test = pd.read_csv('lyrics-test-hidden.csv')

In [71]:
df_train.head()

## Bag of words

The first step is to write functions to produce a bag-of-words representation of each input song (using counts). For this data set, much of the work is already done: each song is already lower-cased, tokenized appropriately (modulo a few errors), and has had its punctuation removed.

* **Deliverable 1.1**: Complete the function `hw2_utils.preproc.bag_of_words`

* **Test**: `nosetests tests/test_preproc.py:test_d1_1_bow`

In [72]:
from hw2_utils import preproc

In [73]:
# run this block as you update the Python file, to trigger a reload in the notebook
reload(preproc);

In [75]:
y_train, x_train = preproc.read_data('lyrics-train.csv')

In [76]:
! nosetests tests_instructor/test_preproc.py:test_d1_1_bow

In [77]:
y_dev, x_dev = preproc.read_data('lyrics-dev.csv')
y_test, x_test = preproc.read_data('lyrics-test-hidden.csv')

## Unseen words

One challenge for text classification is that words may appear in the test data that do not appear in the training data. In order to know how big of a problem this will be, it is useful to do some exploratory data analysis into word frequencies and vocabulary overlap. Since we don't want to look at the test data itself, we'll start by looking into the number of words that appear in `lyrics-dev.csv` but not in `lyrics-train.csv`.

* **Deliverable 1.2**: Implement `hw2_utils.preproc.aggregate_counts`, a function to combine [`Counter`s](https://docs.python.org/3/library/collections.html#collections.Counter) representing document-level bags-of-words into a single `Counter` representing a collection of documents.
* **Deliverable 1.3**: Implement `hw2_utils.preproc.compute_oov`, which returns a list of words that appear in one bag-of-words but not another.
* **Tests**: `tests/test_preproc.py:test_d1_2_agg`, `tests/test_preproc.py:test_d1_3_oov`

In [78]:
from collections import Counter

In [79]:
reload(preproc);

_Tip_: You may find it worthwhile or useuful to use the `%%timeit` Jupyter magic directive to evaluate the performance of your algorithm:

In [80]:
%%timeit
preproc.aggregate_counts(x_dev)

`Counter`s are very useful tools! Note that many common set operations (intersection, etc.) are available, but be warned that they are sometimes _not_ the most performant way to work with Counters. Consult the documentation for details. 

You can see the most items in a `Counter` using the `most_common()` method:

In [81]:
counts_dev = preproc.aggregate_counts(x_dev)
counts_train = preproc.aggregate_counts(x_train)
counts_dev.most_common(5)

In [84]:
%%timeit
preproc.aggregate_counts(x_train)

In [86]:
len(preproc.compute_oov(counts_dev, counts_train))

In [87]:
len(preproc.compute_oov(counts_train, counts_dev))

In [88]:
preproc.oov_rate(counts_dev, counts_train)

Looks like just under 30% of the words in the dev set do not appear in the training set.

## Power laws

As we've discussed, word count distributions typically follow a [power law](https://en.wikipedia.org/wiki/Power_law) distribution.

In practice, this means that a log-log plot of frequency against rank should be linear-ish. Let's see if this holds for our data!

In [89]:
plt.loglog([val for word, val in counts_train.most_common()])
plt.loglog([val for word, val in counts_dev.most_common()])
plt.xlabel('rank')
plt.ylabel('frequency')
plt.legend(['Training', 'Dev'])

Making plots like this is a very helpful "sanity check", to make sure that our data looks like we're expecting it to.

## Pruning the vocabulary

Let's prune the vocabulary to only include words that appear at least _ten_ times in the training data.

* **Deliverable 1.4**: Implement `preproc.prune_vocabulary`
* **Test**: `tests/test_preproc.py:test_d1_4_prune`

In [92]:
reload(preproc);

In [93]:
x_train_pruned, vocab = preproc.prune_vocabulary(counts_train, x_train, 10)
x_dev_pruned, _ = preproc.prune_vocabulary(counts_train, x_dev, 10)
x_test_pruned, _ = preproc.prune_vocabulary(counts_train, x_test, 10)

In [94]:
len(vocab)

In [95]:
print("orig","pruned")
print(len(x_dev[94]), len(x_dev_pruned[94]))
print(sum(x_dev[94].values()), sum(x_dev_pruned[94].values()))

# 2. Linear classification


Now, you'll implement a simple linear classification rule, $\hat{y}=\text{argmax}_y \theta^T f(x,y)$

The functions you write in this section will be used for all the classifiers in the rest of the assignment.

In [96]:
from hw2_utils import clf_base
reload(clf_base);

from hw2_utils import constants
reload(constants);

The first step is to write a feature extraction function. A convenient way to represent a feature function vector $f(x,y)$ for a document $x$ is as a dict, where each value $j$ is the value of the $j$th feature $x_j$ (counts of word occurrence, or whatever it is that you're using as features), and the keys are tuples $(y, x_j)$ indicating _which_ feature-class relationship is being stored.

* **Deliverable 2.1**: Implement the function `make_feature_vector` in `clf_base.py`
* **Test**: `tests/test_classifier.py:test_d2_1_featvec`

_Note_ that you'll also want to make sure to include the bias/offset feature as well, from `hw2_utils.constants.OFFSET`.

Desired output is shown below:

In [97]:
fv = clf_base.make_feature_vector({'test':1, 'case':2}, '1980s')
fv

What is the entire set of labels?

In [98]:
labels = set(y_train) # all possible labels
labels

Now, implement the prediction rule $\hat{y}=\text{argmax}_y \theta^T f(x,y)$.

* **Deliverable 2.2**: Implement the function `predict` in `clf_base.py`.
* **Test**: `tests/test_classifier.py:test_d2_2_predict`

The output should be:

* A predicted label
* The scores of all labels

This function will be called a **lot**, so do try and make it fast. You don't need to do anything too over-the-top, but look for ways to avoid making your code do extra or repeated work. Remember, `%%timeit` can help you explore how different versions of your algorithm perform.

Test your function using the unit test, but also with these simple hand-crafted weights:

In [99]:
from collections import defaultdict

In [100]:
# weight vectors _must_ be defaultdicts as per the specification for predict()
theta_hand = defaultdict(float, 
            {('2000s', 'money'): 0.1,
             ('2000s', 'name'): 0.2,
             ('1980s','tonight'): 0.1,
             ('2000s','man'): 0.1,
             ('1990s','fly'): 0.1,
             ('pre-1980',constants.OFFSET): 0.1

            })


In [101]:
reload(clf_base);

In [102]:
clf_base.predict(x_train_pruned[0], theta_hand, labels)

Now, let's see how well these weights work, by evaluating on the dev set. We'll be using raw classification accuracy here, though that is often not the best choice of metric.

In [103]:
from hw2_utils import evaluation

In [104]:
y_hat = clf_base.predict_all(x_dev_pruned, theta_hand, labels)
print(evaluation.acc(y_hat, y_dev))

Just how imbalanced _are_ our classes, anyway?

In [105]:
np.unique(y_dev, return_counts=True)

In the case of our `dev` data split, there is certainly some class imbalance, though it is not nearly as severe as is often the case in text classification. Using straight-up classification accuracy is not _totally_ unreasonable in this case, though one should make sure to compare it against "random choice" as well as "most-frequent class" baselines.

# 3. Naïve Bayes

You'll now implement a Naïve Bayes classifier, as described in the reading. For a more detailed implementation guide, consult chapter 2 of the Eisenstein notes.

In [106]:
from hw2_utils import naive_bayes
reload(naive_bayes);

* **Deliverable 3.1**: (warmup) implement `get_corpus_counts` in `naive_bayes.py`.
* **Test**: `tests/test_classifier.py:test_d3_1_corpus_counts`

This function should compute unigram word frequency counts for a given label:

In [107]:
eighties_counts = naive_bayes.get_corpus_counts(x_train_pruned, y_train, '1980s');
print(eighties_counts['today'])
print(eighties_counts['yesterday'])

* **Deliverable 3.2**: Implement `estimate_pxy` in `naive_bayes.py`
* **Test**: `tests/test_classifier.py:test_d3_2_pxy`

This function should compute the _smoothed_ multinomial distribution $\log P(x|y)$ for a given label $y$.

_Hint_: Note that this function takes the entire vocabulary as one of its arguments. You'll need to assign a probability even for words that do not appear in documents with label $y$, if they are in-vocabulary.

You can use `get_corpus_counts` in this function if you want to, but you don't have to.

In [108]:
log_pxy = naive_bayes.estimate_pxy(x_train_pruned, y_train, "1980s", 0.1, vocab)

Your probabilities must sum to one! (Or very close- [floating point roundoff](https://en.wikipedia.org/wiki/Round-off_error#Roundoff_error_caused_by_floating-point_arithmetic), dontcha know).

In [110]:
sum(np.exp(list(log_pxy.values())))

Following Emily Bender's wisdom, let's Look 👏 At 👏 Our 👏 Data!

We can examine the log-probabilities of the words from our hand-tuned weights:

In [111]:
{word:log_pxy[word] for (_, word), weight in theta_hand.items() if weight > 0}

What happens if we adjust our smoothing?

In [112]:
log_pxy_smoother = naive_bayes.estimate_pxy(x_train_pruned, y_train, "1980s",10, vocab)

In [113]:
{word:log_pxy_smoother[word] for (_, word), weight in theta_hand.items() if weight > 0}

These are all relatively frequent words, and so their smoothed estimates aren't really all that different. What if we look at a less-common word?

In [114]:
print(eighties_counts['smiles'])
print(log_pxy['smiles'])
print(log_pxy_smoother['smiles'])

This is a much larger change- remember, we're in log-space!

You are now ready to...

* **Deliverable 3.3**: Implement `estimate_nb` in `naive_bayes.py`. This function will estimate label-specific weights ($\theta_{y,j}$ in the notation used by Eisenstein) in the same data format as the weights in `theta_hand` above. 
* **Test**: `tests/test_classifier.py:test_d3_3a_nb`

* The goal is that the score given by `clf_base.predict` is equal to the joint probability $P(x,y)$ as described in Chapter 2 of the Eisenstein text.
* Remember, the offset feature will act as the _prior_ $\log P(y)$, and should not be smoothed

_Tip_: Recall that each weight is supposed to represent the conditional probability of a particular word occurring in a given class.

In [115]:
theta_nb = naive_bayes.estimate_nb(x_train_pruned, y_train, 0.1)

In [116]:
clf_base.predict(x_train_pruned[155], theta_nb, labels)

In [117]:
y_hat = clf_base.predict_all(x_dev_pruned, theta_nb, labels)
print(evaluation.acc(y_hat, y_dev))

As part of the submission of this assignment, you will provide me with your classifier's predictions, which I will validate as part of grading. There are utility functions in `hw2_utils.evaluation` that will automatically produce output in the required format:

In [118]:
evaluation.write_predictions(y_hat, 'nb-dev.preds')

There is also a function to read them back in:

In [119]:
y_hat_dev = evaluation.read_predictions('nb-dev.preds')
evaluation.acc(y_hat_dev, y_dev)

Use this to produce your predicted classifications on the test set:

In [120]:
y_hat_test = clf_base.predict_all(x_test_pruned, theta_nb, labels)
evaluation.write_predictions(y_hat_test, 'nb-test.preds')

Up to now, we have set our smoothing parameter to 0.1. As we saw earlier, though, different amounts of smoothing can have big effects on our learned model weights.

* **Deliverable 3.4**: Implement the `find_best_smoother` function in `naive_bayes.py`, which will grid search over a range of smoothing factors and find the optimal one.
* **Test**: `tests/test_classifier.py:test_d3_4a_nb_best`

Your function should try at least the values set in the `vals` variable below. Once you have obtained the optimal smoothing value, export your predictions on the test subset.

In [122]:
vals = np.logspace(-3, 2, 11)
print(vals)

In [123]:
reload(naive_bayes);

In [124]:
# SBRAICH: NOT IMPLEMENTED
best_smoother, scores = naive_bayes.find_best_smoother(
    x_train_pruned, y_train, x_dev_pruned, y_dev, vals
)

In [ ]:
plt.semilogx(list(scores.keys()), list(scores.values()), 'o-');
plt.xlabel('smoothing factor')
plt.ylabel('dev set accuracy')

**Reflect**:

* What might explain the dramatic drop in accuracy when the smoothing increases from 10 to 30?
* Beore you check, predict whether hte accuracy will continue to significantly drop if you increase it to 100000.

End this section by writing your best predictions out:

In [ ]:
theta_nb_best = naive_bayes.estimate_nb(x_train_pruned, y_train, best_smoother)
y_hat_nb_best = clf_base.predict_all(x_test_pruned, theta_nb_best, labels)
evaluation.write_predictions(y_hat_nb_best, 'nb-best-test.preds')

# 4. Feature Engineering

You can add non-BoW features to your model (in fact, we already have: our bias/offset feature). In order to facilitate additional exploration along these lines, we'll need to change how we're calling our `predict()` function. Up to now, we've been passing it a `Counter` object; now we need to turn that `Counter` into a dictionary, so we can add additional arbitrary features.

## 4.1 Example feature: Token-Type Ratio

One particularly useful additional feature is the token-type ratio:

$$ \frac{\text{length of song in tokens}}{\text{number of distinct types}} = \frac{\sum_m w_m}{\sum_m \delta(w_m > 0)} $$

Intuititively, the more repetitive a song is, the higher its token-type ratio will be. For empty songs (length of zero tokens), this feature should be 0.

* **Deliverable 7.1**: Implement `get_token_type_ratio` in `features.py`
* **Test**: `tests/test_features.py:test_d4_1_token_type_ratio`

In [ ]:
from hw2_utils import features
reload(features);

Token-type ratio for the first five songs:

In [ ]:
[features.get_token_type_ratio(x_train_pruned[i]) for i in range(5)]

## 4.2 Binning

Once we have a token-type ratio, we must include it in our feature vectors. A good way to do that is to _bin_ the feature into several bins:

$\{[(0,1),[1,2),[2,3),[3,4),[4,5),[5,6),[6,\infty)\}$

In other words, for each instance, we will now have seven additional pseudo-features, one per bin. Exactly one will have the value of 1, the rest will be set to zero. Look in `hw2_utils.constants` for feature names for these bins.

* **Deliverable 4.2**: Implement `concat_ttr_binned_feaures` in `features.py`
* **Test**: `tests/test_features.py:test_d4_2_discretize`

In [ ]:
reload(features);
reload(constants);

In [ ]:
features.concat_ttr_binned_features(dict(x_dev_pruned[0]))

## 4.3 Does it affect performance?

If we use this new feature in our prediction, what happens?

In [ ]:
x_train_pruned_with_ttr = [features.concat_ttr_binned_features(dict(x_i)) 
                           for x_i in x_train_pruned]

theta_nb_ttr = naive_bayes.estimate_nb(x_train_pruned_with_ttr, y_train, 0.1)

In [ ]:
x_dev_pruned_with_ttr = [features.concat_ttr_binned_features(dict(x_i)) 
                           for x_i in x_dev_pruned]

y_hat_ttr = clf_base.predict_all(x_dev_pruned_with_ttr, theta_nb_ttr, labels)
print(evaluation.acc(y_hat_ttr, y_dev))

Looks like that feature got us a small performance bump!

_Reflect_: What are some additional features that might help?

# 5. Logistic Regression

In practice, it is rare to build the entire feature-generation pipeline entirely from scratch as we have thus far in this assignment. Scikit-Learn  (`sklearn`) is a massive Python library with many useful off-the-shelf machine learning models, as well as utility functions for feature extraction and transformation. In this section, you will experiment with an off-the-shelf implementation of logistic regression (which we may revisit in more detail in a future assignment).

A `sklearn` workflow typically starts by using one of the automated feature extraction classes. The equivalent to the simple count-based bag-of-words that we've been using thus far is [`sklearn.feature_extraction.text.CountVectorizer`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer). There are several text feature extraction classes, and they have a variety of options in terms of tokenization, punctuation handling, etc.- consult the documentation!

This part of the assignment will require much less coding, and is more exploratory.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

If we already know what we want our vocabulary to be, we can initialize it with an iterable of strings; otherwise, it can build its vocabulary automatically (using its `fit()` method). In this case, we will initialize it to use our already-pruned vocabulary:

In [ ]:
count_vec = CountVectorizer(vocabulary=vocab)

count_vec.transform(df_train.Lyrics[:5])

In [ ]:
X_mat_dev = count_vec.transform(df_dev.Lyrics)
X_mat_train = count_vec.transform(df_train.Lyrics)
X_mat_test = count_vec.transform(df_test.Lyrics)

Once our data is in the `scipy` sparse matrix format, we can use any of the built-in `sklearn` classifiers. For example, naïve Bayes:

In [ ]:
from sklearn.naive_bayes import MultinomialNB

One nice thing about `sklearn` is that all of the classifiers follow a common API:

In [ ]:
nb = MultinomialNB(alpha=0.1)
nb.fit(X_mat_train, df_train.Era)

In [ ]:
y_hat_dev = nb.predict(X_mat_dev)

Another nice thing: `sklearn` has many built-in utility functions for evaluation classifier performance:

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
accuracy_score(y_hat_dev, df_dev.Era)

This is exactly the same accuracy as we saw with our hand-rolled implementation!

We can also print confusion matrices, and more detailed classification output:

In [ ]:
confusion_matrix(y_hat_dev, df_dev.Era)

In [ ]:
print(classification_report(y_hat_dev, df_dev.Era))

In [ ]:
from hw2_utils import clf_sklearn
reload(clf_sklearn);

Next, explore the multinomial logistic regression classifier in `sklearn.linear_model.LogisticRegression`. Pay attention 

* **Deliverable 5.1**: Implement `train_logistic_regression` in `clf_sklearn.py`
* **Test**: `tests/test_clf_sklearn.py:test_d5_1_train_logistic`

What is the classification accuracy for this classifier using bag-of-words features?

It is very common to transform raw count features before using them as input into a classifier. One important transformation is _Term Frequency/Inverse Document Frequency_ (TF/IDF), which attempts to normalize for the fact that certain words have inflated raw counts because they are more commonly used across all documents ("the" is a good example).

`sklearn` has a [`sklearn.feature_extraction.text.TfidfTransformer`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html#sklearn.feature_extraction.text.TfidfTransformer) class, which can transform a count matrix producd using the `CountVectorizer` class we used above. (There is also a `TfidfVectorizer` that can do the counting and transformation in one step).

Adjust your model to use TF/IDF-adjusted word counts. Note that, just as we earlier had to "pin" our vocabulary to that used in the training set, we need to first _fit_ the transformer- in other words, tell it which document frequencies to use. You'll want to make sure those are consistent across train/dev/test splits.

* **Deliverable 5.2**: Implement `transform_tf_idf` in `clf_sklearn.py`, and use it to train and evaluate a new logistic regression model uing the transformed counts.
* **Test**: `tests/test_clf_sklearn.py:test_d5_2_tf_idf`

_Reflect_: What happens to the classification accuracy of the logistic regression model using adjusted feature counts?

In [ ]:
count_vec = CountVectorizer(vocabulary=vocab)
X_train_counts = count_vec.transform(df_train.Lyrics)
X_dev_counts = count_vec.transform(df_dev.Lyrics)
X_test_counts = count_vec.transform(df_test.Lyrics)
(train_tfidf, dev_tfidf, test_dfidf), tf_transformer = clf_sklearn.transform_tf_idf(X_train_counts, X_dev_counts, X_test_counts)

You can ask a `Vectorizer` for an ordered list of feature labels:

In [ ]:
count_vec.get_feature_names()[:5] # list just the first five features

In [ ]:
short_song = count_vec.transform(["you and me and you"])
short_song.toarray() # note the value of feature 0 ("you"):

In [ ]:
shorter_song = count_vec.transform(["you and me and"])
shorter_song.toarray()

In [ ]:
short_song_tfidf = tf_transformer.transform(short_song)
shorter_song_tfidf = tf_transformer.transform(shorter_song)

print(short_song_tfidf.toarray()[0,0])
print(shorter_song_tfidf.toarray()[0,0])

# The End!

Run the `make_submission.sh` script to package up your submission, and turn in your results!